In [14]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
import pandas as pd
import geopandas as gpd
from IPython.display import IFrame
import pathlib

%matplotlib inline

print(ox.__version__)

# turn response caching on and turn on logging to your terminal window
ox.config(log_console=True, use_cache=True)

1.0.1


In [5]:
#read the files from the Colonias folder using geopandas just as if we were using only pandas
colonias = gpd.read_file('input_data/coloniasmexico/Colonias/')

#query to filter only the colonias from Merida municipality of Yucatan state
colonias_yucatan_merida = colonias[(colonias['ST_NAME']=='YUCATAN') & (colonias['MUN_NAME'] == 'MÉRIDA')]

In [5]:
colonias_yucatan_merida.head(3)

,OBJECTID,POSTALCODE,ST_NAME,MUN_NAME,SETT_NAME,SETT_TYPE,AREA,Shape_Leng,Shape_Area,geometry
56515,56516,97000,YUCATAN,MÉRIDA,BARRIO SAN SEBASTIAN,FRACCIONAMIENTO,724836804.0,0.028660,0.000038,"POLYGON Z ((-89.63250 20.96072 0.00000, -89.63..."
56516,56517,97000,YUCATAN,MÉRIDA,BARRIO SANTA ANA,FRACCIONAMIENTO,724836804.0,0.031112,0.000051,"POLYGON Z ((-89.62424 20.97568 0.00000, -89.62..."
56517,56518,97000,YUCATAN,MÉRIDA,BARRIO SANTIAGO,FRACCIONAMIENTO,724836804.0,0.023338,0.000034,"POLYGON Z ((-89.62914 20.96978 0.00000, -89.62..."


In [15]:
# save/load graph as a graphml file: this is the best way to save your model
# for subsequent work later
# if you want to work with your model in gephi, use gephi compatibility mode

file = pathlib.Path('./networks/merida-kanasin-road.graphml')

if file.exists():
    G = ox.load_graphml(filepath)
else:
    places = [{'city' : 'Merida',
             'state' : 'Yucatan',
             'country' : 'Mexico'},
             {'city' : 'Kanasin',
             'state' : 'Yucatan',
             'country' : 'Mexico'}]
    G = ox.graph_from_place(places, network_type='drive')
    ox.save_graphml(G, filepath=file, gephi=False)

In [17]:
#extract nodes as a geodataframe
nodes = ox.graph_to_gdfs(G, edges=False)
nodes.head()

,y,x,street_count,highway,geometry
osmid,,,,,
286974743,20.924688,-89.560150,4,NaN,POINT (-89.56015 20.92469)
287020194,20.896659,-89.647519,3,NaN,POINT (-89.64752 20.89666)
287020195,20.908412,-89.625401,3,NaN,POINT (-89.62540 20.90841)
287020196,20.914672,-89.616335,3,NaN,POINT (-89.61633 20.91467)
287020208,21.042145,-89.610114,3,NaN,POINT (-89.61011 21.04214)


In [48]:
# for each colonia polygon, find intersecting nodes then induce subgraph
j = 0
#subgraphs_colonias = [] #vector for storing subgraphs
stats = [] #vector for storing stats from osmnx
vec = [] #vector for storing stats both basic and extended; only the values
for i in range(colonias_yucatan_merida.size):
    j = j+1
    if j == 10: #only the first 10 colonia; just for testing
        break
        
    #polygon = colonias_yucatan_merida.iloc[i].geometry
    #colonia_name = colonias_yucatan_merida.iloc[i].SETT_NAME
    #mun_name = colonias_yucatan_merida.iloc[i].MUN_NAME
    #ob_id = colonias_yucatan_merida.iloc[i].OBJECTID

    
    try: 
        #try/except for handling empty graphs; 
        #if the graph is empty, NetworkX raises an error and stops the loop;
        #this code ignores the empty graphs in the except statement
        intersecting_nodes = nodes[nodes.intersects(polygon)].index
        G_sub = G.subgraph(intersecting_nodes)
        #subgraphs_colonias.append(G_sub)
        #print(colonia_name+", "+mun_name)
        stat = ox.basic_stats(G_sub)
        extended_stat = ox.extended_stats(G_sub, ecc=True, bc=True, cc=True)
        for key, value in extended_stat.items():
            stat[key] = value #insert the extended stats values
        vec = [stat[elem] for elem in stat if isinstance(stat[elem], dict) == False] #add only the values except the dict type
        stats.append(vec)
        #fig, ax = ox.plot_graph(G_sub)
    except:
        pass

In [49]:
stats

[[30,
  38,
  2.533333333333333,
  28,
  3.466666666666667,
  4427.6089999999995,
  116.51602631578946,
  4288.085,
  119.11347222222223,
  36,
  None,
  None,
  None,
  None,
  1.0001022692745687,
  0.0,
  None,
  None,
  1.2,
  0.018237888316205802,
  0.08735632183908044,
  0.0,
  0.0,
  287661241,
  0.08217985906060121,
  315231835,
  0.006920679467969117,
  1286.675,
  727.998,
  [287661241],
  [6363345517],
  0.000952143919323382,
  0.07032019704433498],
 [46,
  64,
  2.782608695652174,
  38,
  3.1956521739130435,
  6807.298999999999,
  106.36404687499999,
  6392.377999999999,
  112.14698245614034,
  57,
  None,
  None,
  None,
  None,
  1.0006172035802712,
  0.0,
  None,
  None,
  1.3695652173913044,
  0.017569600457330473,
  0.06183574879227054,
  0.0,
  0.0,
  6365185635,
  0.0685027196944954,
  8238384011,
  0.004334442062122589,
  1400.7910000000002,
  876.9119999999999,
  [831384211],
  [6365185631],
  0.000996051130021163,
  0.06639218269653054],
 [30,
  43,
  2.86666666666